说明：  
&ensp;&ensp;本文件使用 Data clean 2 得到的数据集。里面的bank_detail_stats, browse_history_stats,bill_detail_stats 都有不同程度的缺失。当然，目标变量 target的未知数据集部分是缺失的。

<font face="微软雅黑" size=5> Contents： </font>  
1. 数据准备  
2. 过拟合XGB模型  
3. 调参  
&ensp;&ensp;&ensp;&ensp;3.1 自定义网络搜索函数  
4. 交叉验证  
5. 训练最终XGB模型并预测未知样本集  
6. 提交结果


In [45]:
import Custom_Functions as CF    # 此文档记录了部分自定义函数。
import numpy as np
import pandas as pd
import time
from scipy import special
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import covariance
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials,rand
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance
import time
from sklearn.externals import joblib
import pickle
import copy
import matplotlib.pyplot as plt
%matplotlib inline

# 1. 数据准备  
使用 Data Clean 2 得到的数据集，是有缺失、未填充的。将此数据集进行二八分。

In [46]:
# 读取数据集 data1，记为 data

data = pd.read_csv(r'F:\RiskPre2\Data clean 2\data1.csv').set_index("userID")

In [47]:
# 从数据集 data中提取出训练集和未知样本集
# 训练集 data_tr，(55381, 719)。未知样本集 data_te，(13899, 719)

userIDtrain = pd.read_csv(r'F:\RiskPre2\Data clean 1\userIDtrain.csv').values.flatten()
userIDtest = pd.read_csv(r'F:\RiskPre2\Data clean 1\userIDtest.csv').values.flatten()
data_tr = data.loc[userIDtrain,:]
print("shape of data_tr is:",data_tr.shape )
data_te = data.loc[userIDtest,:]
print("shape of data_te is:",data_te.shape )

shape of data_tr is: (55381, 719)
shape of data_te is: (13899, 719)


In [48]:
data_tr.head(2)

,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,bill_AL_cardM__current_minrepay_ratio_max,bill_AL_cardM__current_minrepay_ratio_median,bill_AL_cardM__current_minrepay_ratio_min,bill_AL_cardM__borrow_cash_ratio_max,bill_AL_cardM__borrow_cash_ratio_median,bill_AL_cardM__borrow_cash_ratio_min,bill_card_count,bill_credit_limit_cardsum,loan_time,target
userID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5914855887,0.0
2,0,1,0,0,0,1,0,0,0,0,...,0.897688,0.8637,0.847896,0.944211,0.0,0.0,3.0,61.52325,5914855887,0.0


In [49]:
# 对训练集 data_tr 进行目标变量分离，分成 字段部分 X_tr和 目标变量部分 y_tr。
# 对未知样本集 data_te 进行目标变量分离，分成 字段部分 X_te和 目标变量部分 y_te。


X_tr = data_tr.drop(["target"],axis=1)
y_tr = data_tr.loc[:,"target"]
X_te = data_te.drop(["target"],axis=1)
y_te = data_te.loc[:,"target"]
print("shape of X_tr is:",X_tr.shape)
print("shape of y_tr is:",y_tr.shape)
print("shape of X_te is:",X_te.shape)
print("shape of y_te is:",y_te.shape)

shape of X_tr is: (55381, 718)
shape of y_tr is: (55381,)
shape of X_te is: (13899, 718)
shape of y_te is: (13899,)


In [50]:
X_tr.head(2)

,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,bill_AL_cardM__current_consume_ratio_min,bill_AL_cardM__current_minrepay_ratio_max,bill_AL_cardM__current_minrepay_ratio_median,bill_AL_cardM__current_minrepay_ratio_min,bill_AL_cardM__borrow_cash_ratio_max,bill_AL_cardM__borrow_cash_ratio_median,bill_AL_cardM__borrow_cash_ratio_min,bill_card_count,bill_credit_limit_cardsum,loan_time
userID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5914855887
2,0,1,0,0,0,1,0,0,0,0,...,0.487417,0.897688,0.8637,0.847896,0.944211,0.0,0.0,3.0,61.52325,5914855887


In [51]:
y_tr.head(2)

userID
1    0.0
2    0.0
Name: target, dtype: float64

In [52]:
# 使用 train_test_split 将 X_tr, y_tr 二八划分成训练集 X_tr_train,y_tr_train 和 验证集 X_tr_test, y_tr_test。
# 可见，训练集 44304 个用户，验证集 11077 个用户。
# train_test_split 默认 shuffle数据，所以不必担心。还可以设置 stratify 分层抽样。


X_tr_train, X_tr_test, y_tr_train, y_tr_test = train_test_split(X_tr, y_tr, test_size = 0.2, shuffle=True, stratify=y_tr, random_state = 0)
print("shape of X_tr_train is:",X_tr_train.shape)
print("shape of y_tr_train is:",y_tr_train.shape)
print("shape of X_tr_test is:",X_tr_test.shape)
print("shape of y_tr_test is:",y_tr_test.shape)
scale_pos_weight_value=len(y_tr_train[y_tr_train==0])/len(y_tr_train[y_tr_train==1]) # XGB默认label==0是反类，label==1是正类。
print("negative number/positive number in y_tr_train is:",scale_pos_weight_value)
scale_pos_weight_value1=len(y_tr_test[y_tr_test==0])/len(y_tr_test[y_tr_test==1]) # XGB默认label==0是反类，label==1是正类。
print("negative number/positive number in y_tr_test is:",scale_pos_weight_value1)
# 可见，二八分得的训练集和验证集里的 正负样本比例基本相同，即实现了 stratify.

shape of X_tr_train is: (44304, 718)
shape of y_tr_train is: (44304,)
shape of X_tr_test is: (11077, 718)
shape of y_tr_test is: (11077,)
negative number/positive number in y_tr_train is: 6.7441006816990035
negative number/positive number in y_tr_test is: 6.746153846153846


In [53]:
X_tr_train.head(3)

,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,bill_AL_cardM__current_consume_ratio_min,bill_AL_cardM__current_minrepay_ratio_max,bill_AL_cardM__current_minrepay_ratio_median,bill_AL_cardM__current_minrepay_ratio_min,bill_AL_cardM__borrow_cash_ratio_max,bill_AL_cardM__borrow_cash_ratio_median,bill_AL_cardM__borrow_cash_ratio_min,bill_card_count,bill_credit_limit_cardsum,loan_time
userID,,,,,,,,,,,,,,,,,,,,,
950,0,1,0,0,0,1,0,0,0,0,...,0.0,0.897324,0.895973,0.894623,0.967102,0.483551,0.0,2.0,42.631748,5915633487
52463,0,1,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,60.501599,5931617487
26796,0,1,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,39.814708,5928247887


# 2. 过拟合XGB模型  
 
&ensp;&ensp;调参主要是调过拟合。我觉得应该先尝试最过拟合的可能性，比如树最深等等，看看对训练集和对验证集的auc相差多少。如果训练集的auc很高，验证集的auc很低，说明过拟合。这时可以逐步把与抑制过拟合相关的参数进行调整，使得出现：训练集auc降低，验证集auc提高的效果。   
&ensp;&ensp;这里，对整个训练集X_tr_train，y_tr_train的五折进行学习，学得5个模型，每个模型对自己的训练集和验证集进行预测并算auc，猜测应该训练集的auc很高（过拟合），而测试集的auc很低。

In [54]:
# 对训练集X_tr_train，y_tr_train进行一次性五折划分（此划分和 ModelLR 里的不同）。
# Xt 存储了 5个二维array，分别是5份训练集。  
# Xv 存储了 5个二维array，分别是对应的5份验证集。
# Yt 存储了 5个一维array，分别是对应的5份训练集的标签。
# Yv 存储了 5个一维array，分别是对应的5份验证集的标签。


skf = StratifiedKFold (n_splits=5,shuffle=True,random_state=8)  
# 保证了即shuffle又分层。（其实二八分时已经shuffle过了，此处可以不shuffle，也可以再shuffle一次）
Xt = []
Yt = []
Xv = []
Yv = []
for train_index, validation_index in skf.split(X_tr_train.values,y_tr_train.values):  
    Xt.append(X_tr_train.values[train_index])
    Yt.append(y_tr_train.values[train_index])
    Xv.append(X_tr_train.values[validation_index])
    Yv.append(y_tr_train.values[validation_index])

In [55]:
# 设置 XGB 参数 使过拟合

XGB_param1 =dict(
    max_depth=10,                       # 树的最大深度
    learning_rate=1,                  # 提升树的学习率                     
    n_estimators=1000,                  # 树的个数--1000棵树建立xgboost
    # silent = True,                    # 此为默认设置，不打印running boosting时的message。
    objective='binary:logistic',        # 指定损失函数
    # nthread = -1,（默认）              # 此为默认设置，并行线程数。
    gamma=0,                            # （默认）设置节点划分要达到的loss减小量
    min_child_weight = 1,               # （默认）叶子节点最小实例数据权重和
    max_delta_step = 0,                 # （默认）此为默认设置，每棵树权重改变的最大步长。
    subsample=1,                        # （默认）训练每棵树时实例数据随机采样的比例
    colsample_btree=1,                  # （默认）训练每棵树时特征采样的比例
    colsample_bylevel = 1,              # （默认）此为默认设置，树的每一级的每一次分裂时，对特征的采样比例。
    reg_alpha = 0,                      # （默认此为默认设置，L1正则项系数。
    reg_lambda =0,                      # （默认）此为默认设置，L2正则项系数。
    scale_pos_weight = scale_pos_weight_value,   # 设置正类别样本的权重。解决正反样本个数不平衡的问题。
    base_score = 0.5,                   # （默认）此为默认设置，
    seed = 0,                           # 随机数，因为涉及到随机采样等随机过程。
    missing = np.nan                    # XGB是能处理带缺失值的数据的模型，所以需要指出数据集中用什么代表缺失值。      
    )


In [ ]:
# 对训练集的五折划分，每一折都训练一个XGB模型，每个模型分别对自己的训练集和验证集进行预测并计算 auc。
# Models1 保存了 5 个模型。 Scores1 保存了 5 个模型分别对各自的验证集进行预测算得的 auc。
# 并求 5 个验证集 auc 值 的均值。


timestart = time.time()
Scores1=[]
Models1=[XGBClassifier(**XGB_param1) for i in range(len(Xt))]  # 创建 5 个模型
for i in range(len(Xt)):
    Models1[i].fit(Xt[i],Yt[i], eval_set = [(Xt[i],Yt[i]),( Xv[i],Yv[i])],
                   eval_metric =["logloss","auc"] ,early_stopping_rounds = 10,verbose = True)
    Scores1.append(Models1[i].best_score)
    print("-------------------------------")
print("Time: {:.2f} seconds".format(time.time() - timestart))
print("Scores1 : ", Scores1)
print("The mean auc is :", np.mean(Scores1))

# 代码技巧：我之前是这样做的：Models1=[]，然后在for循环里 modelXGB1.fit数据集，再 Models1.append(modelXGB1)
# 但发现这样做，得到的 Models1 里的五个模型全相同，都是最终的模型。所以我认为，应该在每一轮创建新的模型，
# 而不是同一个模型 modelXGB1 反复 fit 不同的数据。所以上述代码是在每一轮创建新模型。

结论：  
&ensp;&ensp;训练 5 个模型的过程如上，每个模型之间用 长虚线分隔。validation_0是此轮模型对自己的训练集进行预测算得的 logloss 和 auc值。validation_1是此轮模型对自己的验证集进行预测算得的 logloss 和 auc值。每个模型的训练都有好几轮的树，因设置了 early stopping，所以每个模型的最佳轮数会给出。  
&ensp;&ensp;从 5 个模型的训练过程可以看到，5个模型对各自的训练集的预测auc基本可以达到 0.9以上，而对自己的验证集的预测auc维持在 0.7。5个模型的平均auc值为 0.725.  
&ensp;&ensp;如果我们以后能通过超参数调节，让模型对训练集的预测auc降低，让模型对验证集的预测auc升高，就达到了抑制过拟合，提高预测未知的能力的效果。

In [58]:
# 将五折训练出的 五个模型 的具体情况 做成表。
# round 是第几个模型，一共 5 个模型，按计算顺序排列。best_iteration 给出了训练此模型时的最佳轮数（因为设置了 early stopping）。
# train_auc 是此模型对其训练集进行预测计算出的 auc 值，validation_auc 是此模型对其验证集进行预测计算出的 auc 值。

roundXGB=[]  
train_auc=[]  
validation_auc=[]  
best_iteration=[]

for i in range(len(Scores1)):
    
    # round 表是 五折中的第几个模型。一共 5 个模型。
    roundXGB.append(i+1)
    
    # 找出第 i个模型的最佳轮模型在自己训练集上预测时计算得到的auc值。
    train_auc.append(Models1[i].evals_result()["validation_0"]["auc"][ Models1[i].best_iteration])
    
    # 找出第i个模型的最佳轮模型在自己验证集上预测时计算得到的auc值。
    validation_auc.append(Models1[i].evals_result()["validation_1"]["auc"][ Models1[i].best_iteration])
    
    # i个模型的最佳轮是第几轮
    best_iteration.append(Models1[i].best_iteration)

d = {"model_round":roundXGB, "train_auc": train_auc , "validation_auc": validation_auc , "best_iteration": best_iteration }
auc_result = pd.DataFrame(d,index=[1,2,3,4,5])


In [59]:
auc_result

,best_iteration,model_round,train_auc,validation_auc
1,1,1,0.899016,0.726198
2,4,2,0.951195,0.729715
3,5,3,0.954552,0.737586
4,0,4,0.852321,0.716697
5,1,5,0.905869,0.716801


# 3. 调参  
  
1. 因为之前试过GridSearchCV一次调节 6 个超参数，每个超参数都有若干取值，但计算太慢（几天时间也没有算出）。所以这里只做简单调参。
2. 我觉得只调3个参数：learning_rate. max_depth. subsample. 其它都使用默认值。（因为我觉得很多参数在防止过拟合方面也是相互影响的）
3. 调参测量为：先把 learning_rate 维持在较高值，subsample=0.8，然后调整 max_depth。然后，固定learning_rate和max_depth，调整 subsample。然后，固定subsample和max_depth，调整learning_rate。

In [60]:
# 固定参数

XGB_param2 = dict(                 
                      
    n_estimators=1000,                  # 树的个数--1000棵树建立xgboost
    # silent = True,                    # 此为默认设置，不打印running boosting时的message。
    objective='binary:logistic',        # 指定损失函数
    # nthread = -1,（默认）              # 此为默认设置，并行线程数。
    gamma=0,                            # （默认）设置节点划分要达到的loss减小量
    min_child_weight = 1,               # （默认）叶子节点最小实例数据权重和
    max_delta_step = 0,                 # （默认）此为默认设置，每棵树权重改变的最大步长。
    colsample_btree=1,                  # （默认）训练每棵树时特征采样的比例
    colsample_bylevel = 1,              # （默认）此为默认设置，树的每一级的每一次分裂时，对特征的采样比例。
    reg_alpha = 0,                      # （默认此为默认设置，L1正则项系数。
    reg_lambda = 1,                     # （默认）此为默认设置，L2正则项系数。
    scale_pos_weight = scale_pos_weight_value,   # 设置正类别样本的权重。解决正反样本个数不平衡的问题。
    base_score = 0.5,                   # （默认）此为默认设置，
    seed = 1,                           # 随机数，因为涉及到随机采样等随机过程。
    missing = np.nan                    # XGB是能处理带缺失值的数据的模型，所以需要指出数据集中用什么代表缺失值。      
    )

In [61]:
# 因为用 GridSearchCV 对 XGBOOST 的超参数进行交叉验证实在太慢，所以我自定义了一个网格搜索函数 myGridSearchCV1 .
# 可以实现对一个超参数的若干可能取值的 5折 交叉验证。
# 参数：XGB_param是XGB模型的固定参数，param_grid_name 是XGB模型待交叉验证的超参数的名字（字符串），
# param_grid_values 是此超参数的可能取值（list）
# 此函数只能检测一个超参数。
# 返回值：返回一个 dataframe，记录了此超参数的每一个尝试的值 所对应的5个模型的对各自验证集计算的auc的均值。
# 所以，这张表中，mean_auc 最大的那个C值即最优 C。

def myGridSearchCV1(XGB_param,param_grid_name,param_grid_values,Xt,Yt,Xv,Yv):  
    
    records = []
    Info = []     

    for c in param_grid_values:       # 对此超参数的每个值进行尝试
        XGB_param_fix = XGB_param
        XGB_param_fix[param_grid_name] = c # 往固定参数里增添超参数的一个值
                
        Scores=[]
        Models=[XGBClassifier(**XGB_param_fix) for i in range(len(Xt))]  # 创建 5 个模型
        for i in range(len(Xt)):      # 此超参数在一个值下，创建 5 个模型，5个模型分别 fit
            Models[i].fit(Xt[i],Yt[i], eval_set = [(Xt[i],Yt[i]),( Xv[i],Yv[i])],
                       eval_metric =["logloss","auc"] ,early_stopping_rounds = 10,verbose = True)
            
            # 为 5 个模型中的每个收集信息
            k = Models[i].best_iteration  
            r = Models[i].evals_result()
            Infoi = []
            Infoi.append(c)                              # 此超参数值 c 
            Infoi.append(i+1)                            # 此超参数下的第 几 个模型
            Infoi.append(k)                              # 此模型的最佳轮数
            Infoi.append(r["validation_0"]["auc"][k])    # 此模型的最佳轮的训练集 auc
            Infoi.append(r["validation_1"]["auc"][k])    # 此模型的最佳轮的验证集 auc，也就是 Models[i].best_score
            Info.append(Infoi)
            
            Scores.append(Models[i].best_score)         # 此模型的最佳轮的验证集 auc
            print("--------------------------------")
        
        print("When %s is %f, five scores are :" % (param_grid_name,c))
        print(Scores)
        print("--------------------------------")
        print("--------------------------------")
        print("--------------------------------")
        
        records.append([c,np.mean(Scores)])
    
    Info = pd.DataFrame(Info,columns=[param_grid_name,"model_round","best_iteration","train_auc","validation_auc"])
    records = pd.DataFrame(records,columns=[param_grid_name,"mean_auc"]).sort_values(by="mean_auc",ascending=False)
    
    return Info, records

1. 固定 learning_rate=0.5，subsample=0.8，调整 max_depth。

In [62]:
# 固定参数更新

XGB_param3 = XGB_param2.copy()
XGB_param3.update({"learning_rate":0.5,"subsample":0.8})

# 注意，一定要对 XGB_param2 进行复制，再赋给 XGB_param3，这样 更改XGB_param3 不会影响到 XGB_param2。
# 若使用 XGB_param3 = XGB_param2，则对XGB_param3 的更改也同样会反应到 XGB_param2 里。

In [ ]:
# 要调节的超参数 -- max_depth  [1,5,10,15]

timestart=time.time()
Info3_1,result3_1 = myGridSearchCV1(XGB_param3,"max_depth",[1,5,10,15],Xt,Yt,Xv,Yv)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

In [64]:
result3_1

,max_depth,mean_auc
0,1,0.799058
1,5,0.789020
3,15,0.763669
2,10,0.758311


结论：  

max_depth=1时， 对训练集的auc值约为0.81，对验证集的auc均值0.799  
max_depth=5时， 对训练集的auc值约为0.85，对验证集的auc均值0.789  
max_depth=10时，对训练集的auc值约为0.95，对验证集的auc均值0.764  
max_depth=15时，对训练集的auc值约为1，   对验证集的auc均值0.758  
&ensp;&ensp;可见，随着 max_depth 值的增加，得到的模型对训练集逐渐拟合得好，但对验证集的预测结果逐渐变差。  
&ensp;&ensp;我想，是不是因为设置了 early stopping，使得集成树的树个数是依据训练情况自行决定的，导致树的深度max_depth其实不重要？树深度设的低，可以通过增加数个数来弥补？因为max_depth取1,5,10,15这些值时，训练出的模型的树个数是不一样的。


In [ ]:
# 要调节的超参数 -- max_depth  [2,3,4]

timestart=time.time()
Info3_2,result3_2 = myGridSearchCV1(XGB_param3,"max_depth",[2,3,4],Xt,Yt,Xv,Yv)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

结论：  

max_depth=2时， 对训练集的auc值约为0.82，对验证集的auc均值0.800  
max_depth=3时， 对训练集的auc值约为0.84，对验证集的auc均值0.799  
max_depth=4时， 对训练集的auc值约为0.84，对验证集的auc均值0.794  

&ensp;&ensp;结合result3_1，可知，max_depth=2时最优。

2. 固定learning_rate=0.5，max_depth=2，调整 subsample。

In [67]:
# 固定参数更新

XGB_param4 = XGB_param2.copy()
XGB_param4.update({"learning_rate":0.5,"max_depth":2})

In [ ]:
# 要调节的超参数 -- subsample  [0.4,0.6,0.8,1.0]

timestart=time.time()
Info4_1,result4_1 = myGridSearchCV1(XGB_param4,"subsample",[0.4,0.6,0.8,1.0],Xt,Yt,Xv,Yv)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

结论：  

subsample=0.4时， 对训练集的auc值约为0.81，对验证集的auc均值0.791  
subsample=0.6时， 对训练集的auc值约为0.82，对验证集的auc均值0.798  
subsample=0.8时， 对训练集的auc值约为0.82，对验证集的auc均值0.801  
subsample=1.0时， 对训练集的auc值约为0.84，对验证集的auc均值0.805  

&ensp;&ensp;可知，subsample=1.0时最优。

3. 固定subsample=1.0，max_depth=2，调整learning_rate。

In [70]:
# 固定参数更新

XGB_param5 = XGB_param2.copy()
XGB_param5.update({"subsample":1.0,"max_depth":2})

In [ ]:
# 要调节的超参数 -- learning_rate  [0.1,0.2,0.3,0.4,0.5]

timestart=time.time()
Info5_1,result5_1 = myGridSearchCV1(XGB_param5,"learning_rate",[0.1,0.2,0.3,0.4,0.5],Xt,Yt,Xv,Yv)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

In [72]:
result5_1

,learning_rate,mean_auc
2,0.3,0.807298
0,0.1,0.806963
3,0.4,0.805495
1,0.2,0.805277
4,0.5,0.804851


结论：  

learning_rate=0.1时， 对训练集的auc值约为0.76，对验证集的auc均值0.764  
learning_rate=0.2时， 对训练集的auc值约为0.79，对验证集的auc均值0.781  
learning_rate=0.3时， 对训练集的auc值约为0.80，对验证集的auc均值0.788  
learning_rate=0.4时， 对训练集的auc值约为0.81，对验证集的auc均值0.790  
learning_rate=0.5时， 对训练集的auc值约为0.82，对验证集的auc均值0.794  

&ensp;&ensp;可知，learning_rate=0.5时最优。  
&ensp;&ensp;learning_rate 是防止过拟合的，其值越小，应该越保守，太小易欠拟合，太大易过拟合。我觉得这里learning_rate小于0.5时似乎就欠拟合了，可能和 max_depth 有关。

1. 固定subsample=1.0，learning_rate=0.2，调整max_depth。

In [73]:
# 固定参数更新

XGB_param6 = XGB_param2.copy()
XGB_param6.update({"learning_rate":0.2,"subsample":1.0})

In [ ]:
# 要调节的超参数 -- max_depth  [1,5,10,15]

timestart=time.time()
Info6_1,result6_1 = myGridSearchCV1(XGB_param6,"max_depth",[1,5,10,15],Xt,Yt,Xv,Yv)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

In [ ]:
# 要调节的超参数 -- max_depth  [2,3,4]

timestart=time.time()
Info6_2,result6_2 = myGridSearchCV1(XGB_param6,"max_depth",[2,3,4],Xt,Yt,Xv,Yv)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

结论：  
1. 结合 result6_1和result6_2，可见，固定 subsample=1.0，learning_rate=0.2时，max_depth为 3 时最优。验证集 mean_auc 为 0.80568.  
2. 而之前，当固定 subsample=1.0，max_depth=2时，learning_rate为 0.5 时最优。验证集 mean_auc 为 0.794525 。  
3. 所以，我认为，max_depth 和 learning_rate 在防止过拟合方面是相互影响的，所以我想同时调整这两个参数。

## 3.1 自定义网格搜索函数

In [79]:
# 自定义 枚举组合函数Combination_enum 和 网格搜索函数 myGridSearchCV。

# Combination_enum 函数 用来枚举 排列组合的所有可能性。如果你提供了若干个超参数，每个超参数由几个可能取值，则
# 本函数可以枚举出 所有可能的 超参数值组合。
# 参数 d ： 参数字典
# 返回 comb ： 所有可能的组合构成的二维 list，每个子 list 是一个参数组合可能性。
# param_names 是每个子 list 里的值对应的参数名字。顺序是对应的。
# df 是把 d 和 param_name 以 dataframe 的形式展示出来了。
# 举例，Combination_enum({"a":[1,2],"b":[3],"c":[4,5,6]})
# 返回：组合 (4,3,1),(4,3,2),(5,3,1),(5,3,2),(6,3,1),(6,3,2)，按参数反序 c,b,a 的顺序给出。

# 代码提示：其实这个问题本身是很简单的，如果知道参数个数为n，则直接使用一个 n 嵌套 for循环即可。
# 但关键是这里的参数个数 n 不已知，是随机的，看用户提供多少。
# 所以这里的 for 循环就不好编写，我用了 递归 方法达到目的。
# 另外，由于 python 使用的是“引用”的方式，所以如果想要的是对象本身的值，则需要对对象进行复制，
# 这里我使用了更保险的 copy模块的 deepcopy，因为参数不会太多，所以不会耗费太多时间和内存。


def Combination_enum(d):   # d 是参数字典
    
    K=[]                   # K 为参数目录
    V=[]                   # V 为每个参数可能取的值。二维list，与 K 对应。
    for k,v in d.items():
        K.append(k)
        V.append(v)
    
    p = len(K)-1           # p 是最后一个参数的索引
    L=[]                   # L 用来综合得到的组合
    e=[]                   # e 提供初始的组合，是空集合
    
    def fun3(p,e):
        nonlocal L
        if p==-1:
            L.append(copy.deepcopy(e))   # 若 p==-1，说明所有参数都取遍了，
            return                       # 得到了一个组合，可以添加到 L 里了
        for i in range(len(V[p])):
            f = []
            for j in range(len(V[p])):
                f.append(copy.deepcopy(e))
            for j in range(len(V[p])):
                f[j].append(V[p][i])
            fun3(p-1,f[i])
   
    fun3(p,e)      # 采用了递归调用
    
    comb = L
    param_names = [i for i in reversed(K)] # 需要把 K 反序排列
    df = pd.DataFrame(comb,columns = param_names)   
    return comb,param_names, df



# 因为用 GridSearchCV 对 XGBOOST 的超参数进行交叉验证实在太慢，所以我自定义了一个网格搜索函数 myGridSearchCV .
# 可以实现对若干个超参数的所有可能取值组合的 K折 交叉验证。
# 参数：XGB_param是XGB模型的固定参数，Combination_enum 是XGB模型待交叉验证的超参数的“二维list”，每行是一个超参数组合，
# param_names 是超参数的名字，与Combination_enum的每一列对应。Combination_enum和param_names 其实是Combination_enum
# 函数的前两个返回值。
# 也就是 Combination_enum 函数 的返回 dataframe
# X,y是数据集，需为 np.array 形式。
# K 是 K折交叉验证，random_state 是随机数，因为函数内部在划分 K折数据时有随机shuffle。
# 返回值：Info是一个 dataframe，记录了此超参数组合的每一个组合 所对应的 K个模型的详细信息。
# records 也是一个 dataframe，记录了此超参数组合的每一个组合 的 K 个模型对各自验证集的auc评分的均值，并按此值从大到小排列，
# 所以排在最前的就是最佳的超参数组合。

# 注意，这里之所以使用 Combination_enum,param_names，而不是直接使用 Combination_enum函数返回的 df，
# 是因为对 df 的每行进行迭代的时候，会自动改变 dtype，如 int 改为 float，这会使得一些需要为 int 类型的参数在模型 fit
# 时出错，所以 Combination_enum函数被设计为 返回二维列表形式的 comb，保证每个参数值的dtype不变。而 myGridSearchCV 也
# 直接使用 comb，不改变dtype。


def myGridSearchCV2(XGB_param,Combination_enum,param_names,X,y,K,random_state):
    
    # 将提供的数据集（X,y）分成 K 折，得到 训练集 Xt, Yt 和 对应的 验证集 Xv,Yv 。
    # 注意，提供的 X 和 y 必须是 np.array 形式。
    skf = StratifiedKFold(n_splits=K,shuffle=True,random_state=random_state) # 既 shuffle 又 分层抽样。分成 K折。
    Xt = []
    Yt = []
    Xv = []
    Yv = []
    for train_index, validation_index in skf.split(X,y):  
        Xt.append(X_tr_train.values[train_index])
        Yt.append(y_tr_train.values[train_index])
        Xv.append(X_tr_train.values[validation_index])
        Yv.append(y_tr_train.values[validation_index])
 

    # 交叉验证
    records = []  # 记录每个超参数组合下，K个模型对各自验证集的 auc 评分的均值。
    Info = []     # 记录每个超参数组合下，训练的 K 个模型的具体信息，每个模型占一行。
    
    
    for l in  range(len(Combination_enum)):   # 对 Combination_enum的每行，即每个超参数组合进行迭代。
        paramcombdict = {}
        for i in range(len(param_names)):
            paramcombdict[param_names[i]]=Combination_enum[l][i] # paramcombdict 是一个 超参数组合的字典
    
        
        XGB_param_fix = XGB_param            # XGB_param是固定参数，paramcombdict是一个超参数组合
        XGB_param_fix.update(paramcombdict)  # 两者结合构成了完整的 XGB 参数字典
                
        Scores=[]
        Models=[XGBClassifier(**XGB_param_fix) for i in range(len(Xt))]  # 在一套参数下，创建 K 个 XGB模型
        for i in range(len(Xt)):      # 此套参数下，创建 K 个模型，K个模型分别 fit K 折数据
            Models[i].fit(Xt[i],Yt[i], eval_set = [(Xt[i],Yt[i]),( Xv[i],Yv[i])],
                       eval_metric =["logloss","auc"] ,early_stopping_rounds = 10,verbose = True)
            
            # 为 K 个模型中的每个收集信息
            n = Models[i].best_iteration  
            r = Models[i].evals_result()
            Infoi = []
            Infoi.append(paramcombdict)                  # 此超参数值 c 
            Infoi.append(i+1)                            # 此超参数下的第 几 个模型
            Infoi.append(n)                              # 此模型的最佳轮数
            Infoi.append(r["validation_0"]["auc"][n])    # 此模型的最佳轮的训练集 auc
            Infoi.append(r["validation_1"]["auc"][n])    # 此模型的最佳轮的验证集 auc，也就是 Models[i].best_score
            Info.append(Infoi)
            
            Scores.append(Models[i].best_score)         # 此模型的最佳轮的验证集 auc
            print("--------------------------------")
        
        print("When param is %s, five scores are :" % paramcombdict)
        print(Scores)
        print("--------------------------------")
        print("--------------------------------")
        print("--------------------------------")
        
        records.append([paramcombdict,np.mean(Scores)])
    
    Info = pd.DataFrame(Info,columns=["paramcombdict","model_round","best_iteration","train_auc","validation_auc"])
    records = pd.DataFrame(records,columns=["paramcombdict","mean_auc"]).sort_values(by="mean_auc",ascending=False)
    
    return Info, records

1. 固定subsample=1.0，考察 learning_rate 和 max_depth 组合。

In [ ]:
# 准备 max_depth 和 learning_rate 的组合参数值。其每一对组合都会考察。

checkdict = {"max_depth":[2,3,4,5],"learning_rate":[0.1,0.2,0.3,0.4,0.5]}

comb,param_names,checkdf = Combination_enum(checkdict)


In [ ]:
# 交叉验证。对每一对超参数组合进行检查

XGB_param7 = XGB_param2.copy()
XGB_param7.update({"subsample":1.0})

timestart=time.time()
Info, records = myGridSearchCV2(XGB_param7,comb,param_names,X_tr_train.values,y_tr_train.values,K=3,random_state=8)
print("Time: {:.3f} hours".format((time.time() - timestart)/3600))

结论：  
&ensp;&ensp;同时对 learning_rate 和 max_depth 进行网格搜索，得到最佳的超参数组合为：'learning_rate': 0.1, 'max_depth': 3。  
&ensp;&ensp;此时的 对验证集的 预测AUC均值为：0.805289 。  
  
&ensp;&ensp;我们只对 learning_rate，max_depth，subsample 三个超参数进行调参，得到最佳组合为：  
'learning_rate': 0.1, 'max_depth': 3, 'subsample':1.0

In [84]:
# 在得到的最佳超参数下对训练集 X_tr_train, y_tr_train 进行训练，对 验证集 X_tr_test, y_tr_test 进行预测并计算 AUC 值。

In [85]:
# 最佳参数

XGB_param8 = dict(                 
                      
    n_estimators=1000,                  # 树的个数--1000棵树建立xgboost
    # silent = True,                    # 此为默认设置，不打印running boosting时的message。
    objective='binary:logistic',        # 指定损失函数
    # nthread = -1,（默认）              # 此为默认设置，并行线程数。
    gamma=0,                            # （默认）设置节点划分要达到的loss减小量
    min_child_weight = 1,               # （默认）叶子节点最小实例数据权重和
    max_delta_step = 0,                 # （默认）此为默认设置，每棵树权重改变的最大步长。
    colsample_btree=1,                  # （默认）训练每棵树时特征采样的比例
    colsample_bylevel = 1,              # （默认）此为默认设置，树的每一级的每一次分裂时，对特征的采样比例。
    reg_alpha = 0,                      # （默认此为默认设置，L1正则项系数。
    reg_lambda = 1,                     # （默认）此为默认设置，L2正则项系数。
    scale_pos_weight = scale_pos_weight_value,   # 设置正类别样本的权重。解决正反样本个数不平衡的问题。
    base_score = 0.5,                   # （默认）此为默认设置，
    seed = 1,                           # 随机数，因为涉及到随机采样等随机过程。
    missing = np.nan,                   # XGB是能处理带缺失值的数据的模型，所以需要指出数据集中用什么代表缺失值。   
    
    learning_rate = 0.1,         # 这是调参得到的最佳参数，正好，也同时是默认值。
    max_depth = 3,
    subsample = 1.0
    )

In [ ]:
# 对训练集 X_tr_train, y_tr_train 进行训练，对 验证集 X_tr_test, y_tr_test 进行预测并计算 AUC 值。

modelXGB1 = XGBClassifier(**XGB_param8)
timestart = time.time()
modelXGB1.fit(X_tr_train,y_tr_train,eval_set = [(X_tr_train,y_tr_train),(X_tr_test, y_tr_test)],
              eval_metric =["logloss","auc"] ,early_stopping_rounds = 10,verbose = True)
print("Time: {:.2f} seconds".format(time.time() - timestart))

In [87]:
modelXGB1.best_iteration

180

In [88]:
modelXGB1.best_ntree_limit

181

In [89]:
modelXGB1.best_score

0.805896

In [90]:
# 得到一个模型后，用模型对验证集进行概率预测，ks_statistic函数通过对比概率预测结果和验证集的真实类别结果，计算得到 K-S统计量值并返回。
# ks_statistic 函数的参数：y_predicted_proba是对验证集里的每个样本属于正类1的概率预测， y_true是验证集里每个样本的真实类别
# （1正类，0反类）
# 此函数的返回值是 模型对此样本集的预测所计算出的 K-S统计量。K-S统计量的取值范围是0-1，模型的分类效果越好，此值越大。

# 得到一个模型后，用模型对验证集进行概率预测，ks_curve函数可以通过此预测结果画出 K-S曲线。
# ks_curve函数的参数：y_predicted_proba是对验证集里的每个样本属于正类1的概率预测， y_true是验证集里每个样本的真实类别
# （1正类，0反类）
# 此函数返回 K-S 曲线。


def ks_statistic(y_true,y_predicted_proba):
    
    fpr,tpr,thresholds = metrics.roc_curve(y_true,y_predicted_proba,pos_label=1)
    return abs(tpr-fpr).max()


def ks_curve(y_true,y_predicted_proba):
    fpr,tpr,thresholds = metrics.roc_curve(y_true,y_predicted_proba,pos_label=1)
    
    font1 = {'family': 'Calibri','weight': 'normal','size': 18} # 轴标签字体
    font2 = {'family': 'Calibri','weight': 'normal','size': 23} # 图标题字体
    
    fig = plt.figure(figsize = (6,8))
    ax = fig.add_subplot(111)
    ax.plot(thresholds,1.0-tpr,label="overdue",color="navy")
    ax_t = ax.twinx()
    ax_t.plot(thresholds,1.0-fpr,label="normal",color="g")
    ax.plot(thresholds,tpr-fpr,label="K-S",color="darkorange")
    ax.legend(loc=2)
    ax_t.legend(loc=9)
    ax_t.set_xlim(0.0, 1.0)
    ax_t.set_ylim(0.0, 1.0)
    ax.set_title('K-S curve',fontdict=font2 ) 
    ax.set_xlabel('threshold',fontdict=font1,labelpad= 2)  
    ax.set_ylabel('True Positive Rate',fontdict=font1,labelpad= 6)
    ax.set_ylabel('False Positive Rate',fontdict=font1,labelpad= 6)
    ax.set_xlim(0.0, 1.0)
    ax.set_ylim(0.0, 1.0)
       
    plt.show()

In [91]:
# 用 modelXGB1 对验证集X_tr_test,y_tr_test进行预测，用ks_statistic函数计算K-S统计量，值为0.47.
# Data clean 1 中进行XGBOOST特征重要性计算时，得到的ks值是 0.4478，此处比它要高一些。


y_predict_proba1 = modelXGB1.predict_proba(X_tr_test, ntree_limit=181)  # 注意设置 ntree_limit=181，使用最佳轮的集成树
ks_value1 = ks_statistic(np.array(y_tr_test),y_predict_proba1[:,1])
print("ks_value1 is: ",ks_value1)

# 注意，predict_proba会得到每个样本属于每个类别的概率。y_predict_proba是二维array，一行是一个样本，第一列是属于反类0的概率，
# 第二列是属于正类1的概率，与 model.classes_ 对应。y_predict_proba[:,1] 选出了 对每个样本属于正类1的概率的预测结果。

ks_value1 is:  0.47673170615017824


# 4. 交叉验证  
&ensp;&ensp;以上过程是建立在一次性二八分，用八份去通过myGridSearchCV得到最佳超参数，然后在此超参数下对这八份进行学习得到最佳模型，然后用最佳模型对那二份进行预测并计算AUC和KS。  
&ensp;&ensp;一次性二八分会否太单一？我认为可以在此超参数下使用交叉验证，多算几个验证分数。也就是把整个训练集X_tr, y_tr进行五折划分，在此超参数下学得5个模型，这5个模型对各自的验证集进行预测，得到5个auc和ks评分。可以看看这5个评分，算一下均分。

In [95]:
# 本来准备使用 sklearn cross_val_score 进行交叉验证，但是运行太慢。代码如下：
# modelXGB2 = XGBClassifier(**XGB_param8)  # XGB_param8是最佳超参数+固定参数
# scores2 = cross_val_score(modelXGB2, X_tr, y_tr,cv = 5, scoring="roc_auc")  # 或试试cv=10
# print("5Fold scores :", scores2)
# print("5Fold mean score :", np.mean(scores2))


# 准备手动交叉验证。

In [96]:
# 对训练集 X_tr，y_tr进行一次性五折划分。
# Xta 存储了 5个二维array，分别是5份训练集。     # a 表示 all，完整训练集
# Xva 存储了 5个二维array，分别是对应的5份验证集。
# Yta 存储了 5个一维array，分别是对应的5份训练集的标签。
# Yva 存储了 5个一维array，分别是对应的5份验证集的标签。


skf = StratifiedKFold (n_splits=5,shuffle=True,random_state=9)  
# 保证了即shuffle又分层。（其实二八分时已经shuffle过了，此处可以不shuffle，也可以再shuffle一次）
Xta = []
Yta = []
Xva = []
Yva = []
for train_index, validation_index in skf.split(X_tr.values,y_tr.values):  
    Xta.append(X_tr.values[train_index])
    Yta.append(y_tr.values[train_index])
    Xva.append(X_tr.values[validation_index])
    Yva.append(y_tr.values[validation_index])

In [97]:
# 参数使用 最佳超参数 XGB_param8 。

In [ ]:
# 对训练集的五折划分，每一折都训练一个XGB模型，每个模型分别对自己的训练集和验证集进行预测并计算 auc。
# Models2 保存了 5 个模型。 Scores2 保存了 5 个模型分别对各自的验证集进行预测算得的 auc。
# 并求 5 个验证集 auc 值 的均值。


timestart = time.time()
Scores2=[]
Models2=[XGBClassifier(**XGB_param8) for i in range(len(Xta))]  # 创建 5 个模型
for i in range(len(Xta)):
    Models2[i].fit(Xta[i],Yta[i], eval_set = [(Xta[i],Yta[i]),( Xva[i],Yva[i])],
                   eval_metric =["logloss","auc"] ,early_stopping_rounds = 10,verbose = True)
    Scores2.append(Models2[i].best_score)
    print("-------------------------------")
print("Time: {:.2f} seconds".format(time.time() - timestart))
print("Scores1 : ", Scores2)
print("The mean auc is :", np.mean(Scores2))

# 代码技巧：我之前是这样做的：Models2=[]，然后在for循环里 modelXGB2.fit数据集，再 Models2.append(modelXGB2)
# 但发现这样做，得到的 Models2 里的五个模型全相同，都是最终的模型。所以我认为，应该在每一轮创建新的模型，
# 而不是同一个模型 modelXGB2 反复 fit 不同的数据。所以上述代码是在每一轮创建新模型。

In [100]:
# 将五折训练出的 五个模型 的具体情况 做成表。
# model_round 是第几个模型，一共 5 个模型，按计算顺序排列。best_iteration 给出了训练此模型时的最佳轮数
# （因为设置了 early stopping）。
# train_auc 是此模型对其训练集进行预测计算出的 auc 值，validation_auc 是此模型对其验证集进行预测计算出的 auc 值。

roundXGB2=[]  
train_auc2=[]  
validation_auc2=[]  
best_iteration2=[]

for i in range(len(Scores2)):
    
    # round 表是 五折中的第几个模型。一共 5 个模型。
    roundXGB2.append(i+1)
    
    # 找出第 i个模型的最佳轮模型在自己训练集上预测时计算得到的auc值。
    train_auc2.append(Models2[i].evals_result()["validation_0"]["auc"][ Models2[i].best_iteration])
    
    # 找出第i个模型的最佳轮模型在自己验证集上预测时计算得到的auc值。
    validation_auc2.append(Models2[i].evals_result()["validation_1"]["auc"][ Models2[i].best_iteration])
    
    # i个模型的最佳轮是第几轮
    best_iteration2.append(Models2[i].best_iteration)

d = {"model_round":roundXGB2, "train_auc": train_auc2 , "validation_auc": validation_auc2 , "best_iteration": best_iteration2 }
auc_result2 = pd.DataFrame(d,index=[1,2,3,4,5])


In [102]:
np.mean(auc_result2["validation_auc"])

0.8093098

# 5. 训练最终XGB模型并预测未知样本集

In [103]:
# 训练最终XGB模型并预测未知样本集

modelXGB3 = XGBClassifier(**XGB_param8)  # XGB_param8是最佳超参数+固定参数
modelXGB3.fit(X_tr, y_tr)
proba_prediction3 = modelXGB3.predict_proba(X_te)

In [104]:
# 将预测结果整合成表

XGBpre = pd.DataFrame(proba_prediction3[:,1],index=X_te.index,columns=["positive_proba"]).reset_index()
XGBpre.columns = ["userid","probability"]  # 按官网提交示例修改列名

In [106]:
# 保存预测结果

XGBpre.to_csv(r'F:\RiskPre2\result\Data Clean 2\XGBpre.csv',index=False)

In [107]:
# 保存模型

joblib.dump(modelXGB3, r'F:\RiskPre2\result\Data Clean 2\modelXGB3.pkl')

['F:\\RiskPre2\\result\\Data Clean 2\\modelXGB3.pkl']

# 6. 提交结果

In [1]:
# 提交 XGBpre 结果：计算的 ks 值是： 0.40***7 。历史最高。